In [ ]:
import json
import requests
import csv
import configparser
import warnings
import uuid
import pandas as pd
import sqlite3 as sl
import re
import html

# Импортируем субмодуль chardet.universal
from chardet.universaldetector import UniversalDetector
from pprint import pprint
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import time # Импортируем модуль time
from bs4 import BeautifulSoup

In [ ]:
my_data  = pd.read_csv('my_data.secret', sep=',',index_col='parameters')
my_data.astype(str)

devUser=my_data.loc['devUser'].values [0]
devPassword=my_data.loc['devPassword'].values [0]
sferaUrlLogin=my_data.loc['sferaUrlLogin'].values [0]
sferaUrl=my_data.loc['sferaUrl'].values [0]
sferaTestCaseUrl=my_data.loc['sferaTestCaseUrl'].values [0]
sferaTSectionsUrl=my_data.loc['sferaTSectionsUrl'].values [0]
sferaSprintUrl=my_data.loc['sferaSprintUrl'].values [0]
sferaUrlSearch=my_data.loc['sferaUrlSearch'].values [0]





In [ ]:
session = requests.Session()
session.post(sferaUrlLogin, json={"username": devUser, "password": devPassword}, verify=False)

In [ ]:
def getSuperSprintTasks(lst):
    # формируем список спринтов для запроса
    query_sprints = ''
    for sprint in lst:
        query_add = '%2C%27'
        query_end = '%27'
        if query_sprints == '':
            query_add = '%28%27'
        query_sprints += query_add + str(sprint) + query_end
    query_sprints += '%29'
    query = 'area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%2C%27defect%27%29%20and%20sprint%20in%20' + query_sprints + '&size=1000&page=0&attributesToReturn=checkbox%2Cnumber%2Cname%2CactualSprint%2Cpriority%2Cstatus%2Cestimation%2Cspent%2Cassignee%2Cowner%2CdueDate%2CupdateDate%2CcreateDate%2CworkGroup'
    urlQuery = sferaUrlSearch + "?query=" + query
    #urlQuery = 'https://sfera.inno.local/app/tasks/api/v0.1/entities?query=area%3D%27SKOKR%27%20and%20statusCategory%21%3D%27Done%27%20and%20type%20in%20%28%27task%27%29%20and%20sprint%20in%20%28%274246%27%2C%274247%27%2C%274248%27%2C%274249%27%2C%274250%27%2C%274251%27%29&size=1000&page=0'
    response = session.get(urlQuery, verify=False)
    if response.ok != True:
        raise Exception("Error get sprint data " + response)

    tasks = json.loads(response.text)
    return tasks

def checkTasksEstimation(tasks):
    number = []
    name = []
    estimation = []
    sprint = []
    epic = []
    type = []
    for task in tasks['content']:
        number.append(task['number'])
        name.append(task['name'])
        estimation.append(int(task['estimation']/28800))
        sprint.append(task['actualSprint']['name'])
        if 'workGroup' in task:
            type.append(task['workGroup'][0])
        else:
            type.append('Дефект')
        if 'parentNumber' in task:
            epic.append(task['parentNumber'])
        else:
            epic.append('Дефект') 

    tasks_df = pd.DataFrame({
        'number': number,
        'name': name,
        'estimation': estimation,
        'sprint': sprint,
        'epic': epic,
        'type': type
    })
    return tasks_df


tasks = getSuperSprintTasks(['4246', '4247', '4248', '4249', '4250', '4251'])
tasks_df = checkTasksEstimation(tasks)
tasks_df['type'] = tasks_df['type'].astype(str)
tasks_df['name'] = tasks_df['name'].astype(str)


In [ ]:
tasks_df.pivot_table(
    values='estimation',
    index='sprint',
    columns='type',
    aggfunc='sum',
    fill_value=0,
    margins=True, 
    margins_name='Total'
)

In [ ]:
pivot_df = tasks_df.pivot_table(
    values='estimation',
    index=['sprint','epic'],
    columns='type',
    aggfunc='sum',
    fill_value=0,
    margins=True, 
    margins_name='Total'
)
#pivot_df.to_csv('pivot_table_results.csv', index=True)
pivot_df.to_excel('pivot_table_results.xlsx', index=True)

In [ ]:
df = pd.read_json('data/OKR_Group.json') 


In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.to_excel('data/OKR_Group.xlsx', sheet_name='ingredients', index=False)

Собираем HTML страницу для поставки

In [ ]:
# Задаем константы
release = 'OKR_20240623_ATM' # Метка релиза
service = 'skmb-pledge-adapter' # Метка сервиса
new_version = '2402.5.0'

In [ ]:
prod = pd.read_csv('data/prod.csv', header=None,  names=['data'])
def get_service(value):
    lst = str.split(value, ' ')
    return lst[0]

def get_version(value):
    lst = str.split(value, ' ')
    return lst[-1]

prod['service'] = prod['data'].map(get_service)
prod['version'] = prod['data'].map(get_version)
prod.drop(columns=['data'], inplace=True)

In [ ]:
# Формируем запрос
query = 'label%20%3D%20%27' + release + '%27&size=1000&page=0&attributesToReturn=checkbox%2Cnumber%2Cname%2CactualSprint%2Cpriority%2Cstatus%2Cassignee%2Cowner%2CdueDate%2Clabel%2CparentNumber%2Ccomponent'
url= sferaUrlSearch + "?query=" + query
# Делаем запрос задач по фильтру

In [ ]:
response = session.get(url, verify=False)
if response.ok != True:
    raise Exception("Error get sprint data " + response)

tasks = json.loads(response.text)

In [ ]:
tasks

In [ ]:
# Обрабатываем запрос, проходя по всем задачам и формируя списки
component_lst = []
task_directLink_lst = []
prod_version_lst = []
# task_name_lst = []
# task_number = []

counter = 10
for task in tasks['content']:
    for component in task['component']:
        component_name = component['name']
        if component_name not in component_lst:
            component_lst.append(component['name'])           
            template = f"""
    <p>  <macro class=is-locked data-name=SferaTasks id=SferaTasks-mce_{counter} contenteditable=false    data-rtc-uid=16cce9cf-5572-4a48-a85b-3375c3c8ed6d><macro-parameter data-name=query      data-rtc-uid=d2a03405-badc-4db9-b558-c63defb0c191>label = '{release}' and      component='{component_name}'</macro-parameter><macro-parameter data-name=name      data-rtc-uid=299855c1-a21a-4a91-88b3-99539008e3c6>{release}_{component_name}</macro-parameter><macro-parameter      data-name=maxTasks data-rtc-uid=4a9ba1c3-6e18-4939-a108-7890b7347054>20</macro-parameter><macro-parameter      data-name=attributes      data-rtc-uid=150ff77e-2639-48fa-bba0-32dd06b4104f>[{{'name':'Ключ','code':'number'}},{{'name':'Название','code':'name'}},{{'name':'Статус','code':'status'}},{{'name':'Исполнитель','code':'assignee'}}]</macro-parameter><macro-parameter      data-name=createdDate      data-rtc-uid=05fa3e33-7799-4ddc-bc7e-316a518aeeaa>1716579558662</macro-parameter><macro-parameter      data-name=isLocked      data-rtc-uid=3ba10cb8-4865-45c1-aae9-0e8c5437f9c8>false</macro-parameter><macro-rich-text      data-rtc-uid=5dfe052c-765d-4974-8d5d-4d3e356f9bd9></macro-rich-text></macro></p>
    """
            counter+=1
            task_directLink_lst.append(template)
            if component_name in prod['service'].values:
                version = prod.loc[prod['service'] == component_name, 'version'].values[0]
                prod_version_lst.append(version)
            else:
                prod_version_lst.append('')
            


tasks_df = pd.DataFrame({
    'Сервис': component_lst,
    'Задачи в сфере': task_directLink_lst,   
    'Версия поставки Новый цод': new_version,
    'Версия для откатки': prod_version_lst,
    'Требует выкатку связанный сервис': '',
    'Версия еДТО': '',
    'Тест-кейсы': '',
    'БЛОК': '',
    'Комментарии': ''
})

In [ ]:
# Генерируем HTML-код
html_code = tasks_df.to_html(index=False)

# Декодируем HTML-спецсимволы
decoded_html = html.unescape(html_code)
decoded_html = str.replace(decoded_html,'\n','')
decoded_html = str.replace(decoded_html,'\\n','')
decoded_html = str.replace(decoded_html,'"','')
decoded_html = str.replace(decoded_html,"'",'"')
decoded_html = str.replace(decoded_html,'class=sfera-link sfera-task sfera-link-style','class="sfera-link sfera-task sfera-link-style"')


In [ ]:
url = 'https://sfera.inno.local/app/knowledge/api/page/v2/pages'
data = {
"spaceId": "cbbcfa0b-0542-4407-9e49-61c6aa7caf1b",
"parentCid": 426943,
"name": "Test new page 32",
"content": decoded_html
}
response = session.post(url, json=data, verify=False)
if response.ok != True:
    raise Exception("Error creating story " + response)
result = json.loads(response.text)

In [ ]:
url = 'https://sfera.inno.local/app/knowledge/api/page/v1/pages/cid/1258887'
response = session.get(url, verify=False)
if response.ok != True:
    raise Exception("Error get sprint data " + response)

page = json.loads(response.text)


In [ ]:
html_table = page['payload']['content']

# Используем BeautifulSoup для парсинга HTML-таблицы
soup = BeautifulSoup(html_table, 'html.parser')
table = soup.find('table')

# Получаем заголовки столбцов
headers = [header.get_text() for header in table.find_all('th')]

# Инициализируем список для данных
data = []

# Получаем данные из строк таблицы
for row in table.find_all('tr')[1:]:
    row_data = [cell.get_text() for cell in row.find_all('td')]
    data.append(row_data)

# Создаем DataFrame
df = pd.DataFrame(data, columns=headers)



In [ ]:
df1 = tasks_df.copy()
df2 = df.copy()

In [ ]:
# Сравниваем строки по полю 'key'
missing_rows = df1[~df1['Сервис'].isin(df2['Сервис'])]
missing_rows 

In [ ]:
# Перезаписываем df1 значениями из df2 без учета индекса
df1.set_index('Сервис', inplace=True)
df2.set_index('Сервис', inplace=True)

df2.update(df1[['Задачи в сфере']], overwrite=True)

# Сбрасываем индекс для восстановления изначальной структуры
df2.reset_index(inplace=True)

In [ ]:
# Добавляем отсутствующие строки из df1 в df2
df2 = pd.concat([df2, missing_rows], ignore_index=True)

In [ ]:
df2

In [ ]:
# Генерируем HTML-код
html_code = df2.to_html(index=False)

# Декодируем HTML-спецсимволы
decoded_html = html.unescape(html_code)
decoded_html = str.replace(decoded_html,'\n','')
decoded_html = str.replace(decoded_html,'\\n','')
decoded_html = str.replace(decoded_html,'"','')
decoded_html = str.replace(decoded_html,"'",'"')
decoded_html = str.replace(decoded_html,'class=sfera-link sfera-task sfera-link-style','class="sfera-link sfera-task sfera-link-style"')

In [ ]:
url = 'https://sfera.inno.local/app/knowledge/api/page/v2/pages'
data = {
"spaceId": "cbbcfa0b-0542-4407-9e49-61c6aa7caf1b",
"parentCid": 426943,
"name": "Test new page 36",
"content": decoded_html
}
response = session.post(url, json=data, verify=False)
if response.ok != True:
    raise Exception("Error creating story " + response)
result = json.loads(response.text)

In [ ]:
# Пример HTML-таблицы
html_table = """
<table id="myTable">
  <tr>
    <th>Name</th>
    <th>Age</th>
    <th>City</th>
  </tr>
  <tr>
    <td>Ramesh</td>
    <td>25</td>
    <td>Delhi</td>
  </tr>
  <tr>
    <td>Suresh</td>
    <td>30</td>
    <td>Mumbai</td>
  </tr>
</table>
"""

# Парсим HTML-таблицу с помощью BeautifulSoup
soup = BeautifulSoup(html_table, 'html.parser')
table = soup.find('table')

# Создаем новую строку
new_row = soup.new_tag('tr')

# Добавляем новые данные в новую строку
new_data = ['Neha', '28', 'Chennai']
for data in new_data:
    new_cell = soup.new_tag('td')
    new_cell.string = data
    new_row.append(new_cell)

# Вставляем новую строку в таблицу
table.append(new_row)

# Выводим обновленную HTML-таблицу
print(soup.prettify())

In [ ]:
url = 'https://sfera.inno.local/app/knowledge/api/page/v1/pages/cid/1258887'
response = session.get(url, verify=False)
if response.ok != True:
    raise Exception("Error get sprint data " + response)

page = json.loads(response.text)


In [ ]:
html_table = page['payload']['content']

# Используем BeautifulSoup для парсинга HTML-таблицы
soup = BeautifulSoup(html_table, 'html.parser')
table = soup.find('table')



In [ ]:
# Создаем новую строку
new_row = soup.new_tag('tr')

# Добавляем новые данные в новую строку
new_data = ['Neha', '28', 'Chennai']
for data in new_data:
    new_cell = soup.new_tag('td')
    new_cell.string = data
    new_row.append(new_cell)

# Вставляем новую строку в таблицу
table.append(new_row)

# Выводим обновленную HTML-таблицу
print(soup.prettify())